In [1]:
%reload_ext autoreload
%autoreload 2
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy.io.wavfile as wv
import scipy.signal as sig
import wave

from datetime import datetime
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout

from util import util
from util import WavFileParts
from util.logUtil import LOG, LOG_HEADER
from util.confusionMatrix import ConfusionMatrix


Using TensorFlow backend.


### globale settings

In [2]:
classes = ['music', 'voice', 'environment']

macDir = '/Volumes/SAA_DATA/datasets/'
winDir = 'E:/SAA_DATA/'
osDir = winDir
recordingDir = osDir + '/localizationRecordings'

if osDir == winDir:
    storageFolder = 'E:/SAA_DATA/storedData/'
else:
    storageFolder = '/Users/etto/Desktop/storedData/'

baseSrcDir = osDir + 'localizationFiles/20171025AllExtractionsMic4'
orgWavDirs1 = ['G428_0.0_1.4',
              'G527_0.5_1.4',
              'Studio_2.0_4.2'
              ]

orgWavDirs2 = ['G428_2.1_2.4',
              'G527_1.2_5.8',
              'Studio_3.0_2.0'
              ]

orgsG428 = ['G428_0.0_1.4','G428_2.1_2.4']
orgsG527 = ['G527_0.5_1.4','G527_1.2_5.8']
orgsStudio = ['Studio_2.0_4.2','Studio_3.0_2.0']

NFFT = 1024

chunksBaseDir = 'chunks'
rooms = ['Studio', 'G428', 'G527']

### utility functies

In [3]:
def readSoundChunksDynamic(moduleString):
    chunks = importlib.import_module(moduleString).soundChunks
    wfPts = []
    for jsonString in chunks:
        wfPts.append(WavFileParts.WavFilePartFromJson(jsonString))
    return wfPts

def timeFunction(func):
    """
    Aanroep: bijv. fpc = timeFunction(lambda: getFilesPerCategory(srcDir))
    """
    startTime = datetime.now()
    print('Start: ' + startTime.strftime('%H:%M:%S') + '\n=================')

    res = func()
    
    endTime = datetime.now()
    print('\n=================\nEnd: ' + endTime.strftime('%H:%M:%S'))
    print('Time taken: '),
    print(endTime - startTime)
    print()
    
    return res
    
def storeTestData(allSpectros, allClasses, storageName, keyName):
    filename = storageFolder + storageName + '.hd5'
    df = pd.DataFrame(allSpectros)
    df.to_hdf(path_or_buf=filename, key='spectros_' + keyName)

    df = pd.DataFrame(allClasses)
    df.to_hdf(path_or_buf=filename, key='classes_' + keyName)

def retrieveTestData(storageName, keyName):
    filename = storageFolder + storageName + '.hd5'
    specDf = pd.read_hdf(path_or_buf=filename, key='spectros_' + keyName)
    classesDf = pd.read_hdf(path_or_buf=filename, key='classes_' + keyName)
    return specDf.values, classesDf.values
    

### functies tbv trainen

In [4]:
# Maakt een dictionary aan; per categorie alle files (volledig pad) uit de srcDir
# srcDir is een van de orgWavDirs, bijvoorbeeld
#    localizationFiles/20171025AllExtractionsMic4/G428_0.0_1.4
def getFilesPerCategory(srcDir):
    filesPerCategory = {}
    for catDirLong in glob.glob(srcDir + '/*'):
        catDir = catDirLong.replace('\\', '/')
        catDir = catDir.replace(srcDir + '/', '')

        filesPerCategory[catDir] = []
        for filename in glob.glob(catDirLong + '/*'):
            filename = filename.replace('\\','/')
            filesPerCategory[catDir].append(filename)
    return filesPerCategory

def getFilesPerCatFromMultipleDirs(srcDirs, srcDirsBase=''):
    filesPerCat = {}
    for dirName in srcDirs:
        srcDir = srcDirsBase + '/' + dirName
        fpcNw = getFilesPerCategory(srcDir)
        if not filesPerCat:
            filesPerCat = fpcNw
        else:
            for key in filesPerCat:
                filesPerCat[key] += fpcNw[key]
    return filesPerCat
        

In [5]:
# Maakt een dictionary aan; per categorie de spectrogrammen
def getSpectrosFromFilesPerCategory(filesPerCategory):
    spectros = {}
    for clz in classes:
        spectros[clz] = []
        for filename in filesPerCategory[clz]:
            fs, signal = wv.read(filename)
            freq_array, segment_times, spectrogram = sig.spectrogram(x=signal, fs=fs, nfft=NFFT, noverlap=0)
            spectros[clz].append(spectrogram.T)
    return spectros

In [6]:
def getClassLengths(spectrosPerCat):
    clzLengths = {}
    for clz in classes:
        clzLengths[clz] = sum([np.shape(lst)[0] for lst in spectrosPerCat[clz]])
    return clzLengths

In [7]:
# verwacht invoer van getSpectrosFromFilesPerCategory
# levert traindata op (X_train en Y_train)
def createTrainDataFromSpectros(spectrosPerCat, clzLengths):
    X_train = np.concatenate(spectrosPerCat[classes[0]], axis=0)
    for i in range(1, len(classes)):
        nwSpectros = np.concatenate(spectrosPerCat[classes[i]], axis=0)
        X_train = np.concatenate((X_train,nwSpectros), axis=0)
    
    # one-hot encoding voor Y_train
    nrFiles = clzLengths[classes[0]]
    Y_train = np.array((np.ones(nrFiles),np.zeros(nrFiles), np.zeros(nrFiles))).T

    nrFiles = clzLengths[classes[1]]
    Y_train_nw = np.array((np.zeros(nrFiles), np.ones(nrFiles), np.zeros(nrFiles))).T
    Y_train = np.concatenate((Y_train, Y_train_nw),axis=0)

    nrFiles = clzLengths[classes[2]]
    Y_train_nw = np.array((np.zeros(nrFiles), np.zeros(nrFiles), np.ones(nrFiles))).T
    Y_train = np.concatenate((Y_train, Y_train_nw),axis=0)
    
    return X_train, Y_train

### deep learning model met dropout

In [8]:
def create_model(layersizes):
    # create model
    model = Sequential()
    model.add(Dropout(0.2, input_shape=(513,)))
    model.add(Dense(layersizes[0], activation='relu'))
    for lsize in layersizes[1:]:
        model.add(Dropout(0.2))
        model.add(Dense(lsize, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [9]:
def getModelFileName(modelFilePath, baseModelFilename, layers, nrEpochs):
    modelFilename = modelFilePath + baseModelFilename
    for lsize in layers:
        modelFilename = '{}_{}'.format(modelFilename, lsize)
    modelFilename += 'ep{}'.format(nrEpochs)
    modelFilename += '.hd5'
    return modelFilename

In [10]:
def getModelName(baseModelFilename, layers, nrEpochs):
    modelFilename = baseModelFilename
    for lsize in layers:
        modelFilename = '{}_{}'.format(modelFilename, lsize)
    modelFilename += 'ep{}'.format(nrEpochs)
    return modelFilename

In [11]:
def train_model(X_train, Y_train, layers, nrEpochs, modelFilePath, baseModelFilename):
    soundModel = create_model(layers)
    history = timeFunction(lambda: soundModel.fit(X_train,Y_train, epochs=nrEpochs, shuffle=True, verbose=1))
    soundModel.save(getModelFileName(modelFilePath, baseModelFilename, layers, nrEpochs))
    plt.plot(history.history['loss'])
    plt.show()

In [12]:
def evaluate_model(X_test, realClasses, layers, nrEpochs, modelFilePath, baseModelFilename):
    soundModel = load_model(getModelFileName(modelFilePath, baseModelFilename, layers, nrEpochs))

    # predicted classes
    predictions = soundModel.predict(X_test)
    predClasses = predictions.argmax(axis=1)

    matrix = ConfusionMatrix(classes)
    for vals in zip(realClasses, predClasses):
        matrix.add(int(vals[0]), int(vals[1]), 1)
    LOG(matrix.toString(),True)
    LOG('', True)
    LOG(matrix.toF1String(), True)
    return matrix

In [13]:
def train_and_evaluate_per_epoch(X_train, Y_train, realClasses, layers, nrEpochs, modelFilePath, baseModelFilename):
    soundModel = create_model(layers)
    for epNr in range(1, nrEpochs+1):
        LOG('\n*****************\n* Epoch nr {}\n*****************\n'.format(epNr), True)
        soundModel.fit(X_train,Y_train, epochs=1, shuffle=True, verbose=1)
        soundModel.save(getModelFileName(modelFilePath, baseModelFilename, layers, epNr))
        evaluate_model(X_train, realClasses, layers, epNr, modelFilePath, baseModelFilename)

### functies tbv testen

In [14]:
def createAndStoreTestData(wavFileParts, baseDir, fileDate, micNr, storeFilename, keyName):
    allSpectros = np.array([])
    allClasses = np.array([])

    for wfPt in wavFileParts: #type: WavFilePart
        if not 'Gunshot' in wfPt.getSoundType():
            filename = baseDir + '/{:d}_{:d}_mono{:d}.wav'.format(fileDate, wfPt.fileNr, micNr)
            fs, signal = wv.read(filename)

            classNr = classes.index(wfPt.getSoundType().lower())
            for soundChunk in wfPt.getSoundChunks(micNr):
                startFrame = int(soundChunk[0] * fs)
                endFrame = int(soundChunk[1] * fs)

                sigChunk = signal[startFrame: endFrame]
                freq_array, segment_times, spectrogram = sig.spectrogram(x=sigChunk, fs=fs, nfft=NFFT, noverlap=0)
                if len(allSpectros) == 0:
                    allSpectros = spectrogram.T
                else:
                    allSpectros = np.append(allSpectros, spectrogram.T, axis=0)
                allClasses = np.append(allClasses, classNr * np.ones(len(segment_times)))
                
    storeTestData(allSpectros, allClasses, storeFilename, keyName)


In [15]:
def getTrainDataFromFolders(orgWavDirs):
    fpc = getFilesPerCatFromMultipleDirs(orgWavDirs, baseSrcDir)
    spcs = getSpectrosFromFilesPerCategory(fpc)
    clzLengths = getClassLengths(spcs)
    X_train, Y_train = createTrainDataFromSpectros(spcs, clzLengths)
    # real train classes
    realTrainClasses = np.concatenate((np.zeros(clzLengths[classes[0]]), 
                                  np.ones(clzLengths[classes[1]]), 
                                  2*np.ones(clzLengths[classes[2]])))
    return X_train, Y_train, realTrainClasses


### trainen van model obv alle orgwavs

In [16]:
baseModelFilename = '20180820_allOrgs_dropout'
modelFilePath = storageFolder
logPrefix = 'Alle Orgs met dropout'
orgWavDirs = orgWavDirs1 + orgWavDirs2
layersList = [[100, 20], [400, 250, 100, 20], [400, 300, 200, 100, 50, 20, 10], 
              [450, 400, 350, 300, 250, 200, 150, 100, 50, 21]]
nrEpochs = 5
nrsEpochs = [1,2,3,4,5]

In [17]:
# Train voor alle lagen
X_train, Y_train, realTrainClasses = getTrainDataFromFolders(orgWavDirs)
for layers in layersList:
    LOG_HEADER(logPrefix + ', lagen: {}'.format(str(layers)), True)
    train_and_evaluate_per_epoch(X_train, Y_train, realTrainClasses, layers, nrEpochs, modelFilePath, baseModelFilename)

####################################
#
# Alle Orgs met dropout, lagen: [100, 20]
#
####################################


*****************
* Epoch nr 1
*****************

Epoch 1/1
1108305/1108305 [==============================] - 125s 113us/step - loss: 0.2079 - acc: 0.9236
                   m       v       e |  sens   acc
music         343808   21712    1016 |  0.94  0.97
voice          11945  346522    8256 |  0.94  0.95
environment     1125   11384  362537 |  0.97  0.98
--------------------------------------
prec            0.96    0.91    0.98

F1 overall: 0.95
F1 music: 0.95
F1 voice: 0.93
F1 environment: 0.97


*****************
* Epoch nr 2
*****************

Epoch 1/1
1108305/1108305 [==============================] - 127s 114us/step - loss: 0.1440 - acc: 0.9465
                   m       v       e |  sens   acc
music         343059   22761     716 |  0.94  0.97
voice           7627  351585    7511 |  0.96  0.96
environment      881    8608  365557 |  0.97  0.98
-----------

Epoch 1/1
1108305/1108305 [==============================] - 489s 441us/step - loss: 0.2188 - acc: 0.9240
                   m       v       e |  sens   acc
music         334266   30408    1862 |  0.91  0.97
voice           5265  335992   25466 |  0.92  0.94
environment      630    2973  371443 |  0.99  0.97
--------------------------------------
prec            0.98    0.91    0.93

F1 overall: 0.94
F1 music: 0.95
F1 voice: 0.91
F1 environment: 0.96


*****************
* Epoch nr 2
*****************

Epoch 1/1
1108305/1108305 [==============================] - 489s 441us/step - loss: 0.1791 - acc: 0.9394
                   m       v       e |  sens   acc
music         338851   26762     923 |  0.92  0.97
voice           6753  352116    7854 |  0.96  0.95
environment     2542    9164  363340 |  0.97  0.98
--------------------------------------
prec            0.97    0.91    0.98

F1 overall: 0.95
F1 music: 0.95
F1 voice: 0.93
F1 environment: 0.97


*****************
* Epoch nr 3
*****

In [18]:
# Test alle modellen

# model parameters
modelFilePath = storageFolder
layerss = layersList

# test data
micNr = 4
testFileNames = ['testData_G428', 'testData_G527', 'testData_Studio']
keyname = 'mic{}'.format(micNr)

for testFileName in testFileNames:
    for layers in layerss:
        for nrEpochs in nrsEpochs:
            testSpecs, testClasses = retrieveTestData(testFileName, keyname)

            LOG_HEADER(getModelName(baseModelFilename, layers, nrEpochs) 
                       + '\n# ' + keyname
                       + '\n# ' + testFileName,
                       True)
            matrix = evaluate_model(testSpecs, testClasses, layers, nrEpochs, modelFilePath, baseModelFilename)


####################################
#
# 20180820_allOrgs_dropout_100_20ep1
# mic4
# testData_G428
#
####################################

                   m       v       e |  sens   acc
music          19994     616      29 |  0.97  0.96
voice            999   13177     260 |  0.91  0.96
environment       49      19   13157 |  0.99  0.99
--------------------------------------
prec            0.95    0.95    0.98

F1 overall: 0.96
F1 music: 0.96
F1 voice: 0.93
F1 environment: 0.99

####################################
#
# 20180820_allOrgs_dropout_100_20ep2
# mic4
# testData_G428
#
####################################

                   m       v       e |  sens   acc
music          19903     713      23 |  0.96  0.97
voice            640   13603     193 |  0.94  0.97
environment       38      30   13157 |  0.99  0.99
--------------------------------------
prec            0.97    0.95    0.98

F1 overall: 0.97
F1 music: 0.97
F1 voice: 0.95
F1 environment: 0.99

######################

                   m       v       e |  sens   acc
music          19821     786      32 |  0.96  0.97
voice            431   13846     159 |  0.96  0.97
environment       48      49   13128 |  0.99  0.99
--------------------------------------
prec            0.98    0.94    0.99

F1 overall: 0.97
F1 music: 0.97
F1 voice: 0.95
F1 environment: 0.99

####################################
#
# 20180820_allOrgs_dropout_450_400_350_300_250_200_150_100_50_21ep4
# mic4
# testData_G428
#
####################################

                   m       v       e |  sens   acc
music          19791     832      16 |  0.96  0.97
voice            388   13885     163 |  0.96  0.97
environment       50      50   13125 |  0.99  0.99
--------------------------------------
prec            0.98    0.94    0.99

F1 overall: 0.97
F1 music: 0.97
F1 voice: 0.95
F1 environment: 0.99

####################################
#
# 20180820_allOrgs_dropout_450_400_350_300_250_200_150_100_50_21ep5
# mic4
# testData_G428


                   m       v       e |  sens   acc
music           8837      75       5 |  0.99  0.98
voice            251    2615       0 |  0.91  0.98
environment        6       1    2484 |  1.00  1.00
--------------------------------------
prec            0.97    0.97    1.00

F1 overall: 0.97
F1 music: 0.98
F1 voice: 0.94
F1 environment: 1.00

####################################
#
# 20180820_allOrgs_dropout_450_400_350_300_250_200_150_100_50_21ep1
# mic4
# testData_G527
#
####################################

                   m       v       e |  sens   acc
music           8793     117       7 |  0.99  0.98
voice            112    2743      11 |  0.96  0.98
environment        5       0    2486 |  1.00  1.00
--------------------------------------
prec            0.99    0.96    0.99

F1 overall: 0.98
F1 music: 0.99
F1 voice: 0.96
F1 environment: 1.00

####################################
#
# 20180820_allOrgs_dropout_450_400_350_300_250_200_150_100_50_21ep2
# mic4
# testData_G527


                   m       v       e |  sens   acc
music          55541    7142     912 |  0.87  0.94
voice            319   36751     834 |  0.97  0.94
environment       41     358   41938 |  0.99  0.99
--------------------------------------
prec            0.99    0.83    0.96

F1 overall: 0.93
F1 music: 0.93
F1 voice: 0.89
F1 environment: 0.98

####################################
#
# 20180820_allOrgs_dropout_400_300_200_100_50_20_10ep3
# mic4
# testData_Studio
#
####################################

                   m       v       e |  sens   acc
music          56328    6300     967 |  0.89  0.94
voice           1003   35837    1064 |  0.95  0.94
environment       36     206   42095 |  0.99  0.98
--------------------------------------
prec            0.98    0.85    0.95

F1 overall: 0.93
F1 music: 0.93
F1 voice: 0.89
F1 environment: 0.97

####################################
#
# 20180820_allOrgs_dropout_400_300_200_100_50_20_10ep4
# mic4
# testData_Studio
#
####################

### trainen van model obv: G428

In [19]:
baseModelFilename = '20180820_orgG428_dropout'
modelFilePath = storageFolder
logPrefix = 'Orgs G428 met dropout'
orgWavDirs = orgsG428
layersList = [[100, 20], [400, 250, 100, 20], [400, 300, 200, 100, 50, 20, 10], 
              [450, 400, 350, 300, 250, 200, 150, 100, 50, 21]]
nrEpochs = 5
nrsEpochs = [1,2,3,4,5]

In [20]:
# Train voor alle lagen
X_train, Y_train, realTrainClasses = getTrainDataFromFolders(orgWavDirs)
for layers in layersList:
    LOG_HEADER(logPrefix + ', lagen: {}'.format(str(layers)), True)
    train_and_evaluate_per_epoch(X_train, Y_train, realTrainClasses, layers, nrEpochs, modelFilePath, baseModelFilename)

####################################
#
# Orgs G428 met dropout, lagen: [100, 20]
#
####################################


*****************
* Epoch nr 1
*****************

Epoch 1/1
375667/375667 [==============================] - 112s 298us/step - loss: 0.2014 - acc: 0.9263
                   m       v       e |  sens   acc
music         112205    8948     188 |  0.92  0.97
voice           1864  116874    2895 |  0.96  0.95
environment      162    3914  128617 |  0.97  0.98
--------------------------------------
prec            0.98    0.90    0.98

F1 overall: 0.95
F1 music: 0.95
F1 voice: 0.93
F1 environment: 0.97


*****************
* Epoch nr 2
*****************

Epoch 1/1
375667/375667 [==============================] - 105s 279us/step - loss: 0.1260 - acc: 0.9535
                   m       v       e |  sens   acc
music         116502    4628     211 |  0.96  0.98
voice           3780  115537    2316 |  0.95  0.96
environment      151    2816  129726 |  0.98  0.99
---------------

375667/375667 [==============================] - 208s 555us/step - loss: 0.1932 - acc: 0.9328
                   m       v       e |  sens   acc
music         111794    9207     340 |  0.92  0.97
voice           1311  117315    3007 |  0.96  0.96
environment       97    2362  130234 |  0.98  0.98
--------------------------------------
prec            0.99    0.91    0.97

F1 overall: 0.96
F1 music: 0.95
F1 voice: 0.94
F1 environment: 0.98


*****************
* Epoch nr 2
*****************

Epoch 1/1
375667/375667 [==============================] - 202s 539us/step - loss: 0.1441 - acc: 0.9514
                   m       v       e |  sens   acc
music         110114   11169      58 |  0.91  0.97
voice            435  118843    2355 |  0.98  0.96
environment      248    1782  130663 |  0.98  0.99
--------------------------------------
prec            0.99    0.90    0.98

F1 overall: 0.96
F1 music: 0.95
F1 voice: 0.94
F1 environment: 0.98


*****************
* Epoch nr 3
*****************



In [21]:
# Test alle modellen

# model parameters
modelFilePath = storageFolder
layerss = layersList

# test data
micNr = 4
testFileNames = ['testData_G428', 'testData_G527', 'testData_Studio']
keyname = 'mic{}'.format(micNr)

for testFileName in testFileNames:
    for layers in layerss:
        for nrEpochs in nrsEpochs:
            testSpecs, testClasses = retrieveTestData(testFileName, keyname)

            LOG_HEADER(getModelName(baseModelFilename, layers, nrEpochs) 
                       + '\n# ' + keyname
                       + '\n# ' + testFileName,
                       True)
            matrix = evaluate_model(testSpecs, testClasses, layers, nrEpochs, modelFilePath, baseModelFilename)


####################################
#
# 20180820_orgG428_dropout_100_20ep1
# mic4
# testData_G428
#
####################################

                   m       v       e |  sens   acc
music          19282    1303      54 |  0.93  0.97
voice            227   13902     307 |  0.96  0.96
environment       25      53   13147 |  0.99  0.99
--------------------------------------
prec            0.99    0.91    0.97

F1 overall: 0.96
F1 music: 0.96
F1 voice: 0.94
F1 environment: 0.98

####################################
#
# 20180820_orgG428_dropout_100_20ep2
# mic4
# testData_G428
#
####################################

                   m       v       e |  sens   acc
music          19402    1171      66 |  0.94  0.96
voice            661   13497     278 |  0.93  0.95
environment       20      96   13109 |  0.99  0.99
--------------------------------------
prec            0.97    0.91    0.97

F1 overall: 0.95
F1 music: 0.95
F1 voice: 0.92
F1 environment: 0.98

######################

                   m       v       e |  sens   acc
music          18783    1726     130 |  0.91  0.96
voice             54   14214     168 |  0.98  0.96
environment       25     176   13024 |  0.98  0.99
--------------------------------------
prec            1.00    0.88    0.98

F1 overall: 0.95
F1 music: 0.95
F1 voice: 0.93
F1 environment: 0.98

####################################
#
# 20180820_orgG428_dropout_450_400_350_300_250_200_150_100_50_21ep4
# mic4
# testData_G428
#
####################################

                   m       v       e |  sens   acc
music          19322    1245      72 |  0.94  0.97
voice            161   14090     185 |  0.98  0.96
environment       22     138   13065 |  0.99  0.99
--------------------------------------
prec            0.99    0.91    0.98

F1 overall: 0.96
F1 music: 0.96
F1 voice: 0.94
F1 environment: 0.98

####################################
#
# 20180820_orgG428_dropout_450_400_350_300_250_200_150_100_50_21ep5
# mic4
# testData_G428


                   m       v       e |  sens   acc
music           8593     307      17 |  0.96  0.94
voice            558    2308       0 |  0.81  0.94
environment        1       5    2485 |  1.00  1.00
--------------------------------------
prec            0.94    0.88    0.99

F1 overall: 0.93
F1 music: 0.95
F1 voice: 0.84
F1 environment: 1.00

####################################
#
# 20180820_orgG428_dropout_450_400_350_300_250_200_150_100_50_21ep1
# mic4
# testData_G527
#
####################################

                   m       v       e |  sens   acc
music           8212     692      13 |  0.92  0.94
voice             82    2780       4 |  0.97  0.95
environment        0       2    2489 |  1.00  1.00
--------------------------------------
prec            0.99    0.80    0.99

F1 overall: 0.94
F1 music: 0.95
F1 voice: 0.88
F1 environment: 1.00

####################################
#
# 20180820_orgG428_dropout_450_400_350_300_250_200_150_100_50_21ep2
# mic4
# testData_G527


                   m       v       e |  sens   acc
music          40424   20649    2522 |  0.64  0.84
voice             13   36309    1582 |  0.96  0.82
environment        8    3130   39199 |  0.93  0.95
--------------------------------------
prec            1.00    0.60    0.91

F1 overall: 0.81
F1 music: 0.78
F1 voice: 0.74
F1 environment: 0.92

####################################
#
# 20180820_orgG428_dropout_400_300_200_100_50_20_10ep3
# mic4
# testData_Studio
#
####################################

                   m       v       e |  sens   acc
music          40990   20744    1861 |  0.64  0.84
voice            335   36481    1088 |  0.96  0.83
environment       10    2786   39541 |  0.93  0.96
--------------------------------------
prec            0.99    0.61    0.93

F1 overall: 0.82
F1 music: 0.78
F1 voice: 0.75
F1 environment: 0.93

####################################
#
# 20180820_orgG428_dropout_400_300_200_100_50_20_10ep4
# mic4
# testData_Studio
#
####################

### trainen van model obv: G527

In [22]:
baseModelFilename = '20180820_orgG527_dropout'
modelFilePath = storageFolder
logPrefix = 'Orgs G527 met Dropout'
orgWavDirs = orgsG527
layersList = [[100, 20], [400, 250, 100, 20], [400, 300, 200, 100, 50, 20, 10], 
              [450, 400, 350, 300, 250, 200, 150, 100, 50, 21]]
nrEpochs = 5
nrsEpochs = [1,2,3,4,5]

In [23]:
# Train voor alle lagen
X_train, Y_train, realTrainClasses = getTrainDataFromFolders(orgWavDirs)
for layers in layersList:
    LOG_HEADER(logPrefix + ', lagen: {}'.format(str(layers)), True)
    train_and_evaluate_per_epoch(X_train, Y_train, realTrainClasses, layers, nrEpochs, modelFilePath, baseModelFilename)

####################################
#
# Orgs G527 met Dropout, lagen: [100, 20]
#
####################################


*****************
* Epoch nr 1
*****************

Epoch 1/1
378662/378662 [==============================] - 186s 492us/step - loss: 0.2096 - acc: 0.9233
                   m       v       e |  sens   acc
music         109398   12609     298 |  0.89  0.96
voice            544  118540    3530 |  0.97  0.95
environment      217    3478  130048 |  0.97  0.98
--------------------------------------
prec            0.99    0.88    0.97

F1 overall: 0.94
F1 music: 0.94
F1 voice: 0.92
F1 environment: 0.97


*****************
* Epoch nr 2
*****************

Epoch 1/1
378662/378662 [==============================] - 173s 456us/step - loss: 0.1399 - acc: 0.9489
                   m       v       e |  sens   acc
music         116881    5258     166 |  0.96  0.97
voice           4233  114582    3799 |  0.93  0.96
environment      266    2085  131392 |  0.98  0.98
---------------

378662/378662 [==============================] - 307s 812us/step - loss: 0.2070 - acc: 0.9286
                   m       v       e |  sens   acc
music         111664   10426     215 |  0.91  0.97
voice           1138  119284    2192 |  0.97  0.95
environment      222    4429  129092 |  0.97  0.98
--------------------------------------
prec            0.99    0.89    0.98

F1 overall: 0.95
F1 music: 0.95
F1 voice: 0.93
F1 environment: 0.97


*****************
* Epoch nr 2
*****************

Epoch 1/1
378662/378662 [==============================] - 292s 772us/step - loss: 0.1585 - acc: 0.9463
                   m       v       e |  sens   acc
music         116150    5791     364 |  0.95  0.98
voice           2990  115236    4388 |  0.94  0.96
environment      192    1353  132198 |  0.99  0.98
--------------------------------------
prec            0.97    0.94    0.97

F1 overall: 0.96
F1 music: 0.96
F1 voice: 0.94
F1 environment: 0.98


*****************
* Epoch nr 3
*****************



In [24]:
# Test alle modellen

# model parameters
modelFilePath = storageFolder
layerss = layersList

# test data
micNr = 4
testFileNames = ['testData_G428', 'testData_G527', 'testData_Studio']
keyname = 'mic{}'.format(micNr)

for testFileName in testFileNames:
    for layers in layerss:
        for nrEpochs in nrsEpochs:
            testSpecs, testClasses = retrieveTestData(testFileName, keyname)

            LOG_HEADER(getModelName(baseModelFilename, layers, nrEpochs) 
                       + '\n# ' + keyname
                       + '\n# ' + testFileName,
                       True)
            matrix = evaluate_model(testSpecs, testClasses, layers, nrEpochs, modelFilePath, baseModelFilename)


####################################
#
# 20180820_orgG527_dropout_100_20ep1
# mic4
# testData_G428
#
####################################

                   m       v       e |  sens   acc
music          19284    1305      50 |  0.93  0.96
voice            257   13726     453 |  0.95  0.96
environment      176      65   12984 |  0.98  0.98
--------------------------------------
prec            0.98    0.91    0.96

F1 overall: 0.95
F1 music: 0.96
F1 voice: 0.93
F1 environment: 0.97

####################################
#
# 20180820_orgG527_dropout_100_20ep2
# mic4
# testData_G428
#
####################################

                   m       v       e |  sens   acc
music          19833     765      41 |  0.96  0.95
voice           1194   12820     422 |  0.89  0.95
environment      334      92   12799 |  0.97  0.98
--------------------------------------
prec            0.93    0.94    0.97

F1 overall: 0.94
F1 music: 0.94
F1 voice: 0.91
F1 environment: 0.97

######################

                   m       v       e |  sens   acc
music          19732     881      26 |  0.96  0.96
voice            548   13638     250 |  0.94  0.96
environment      350     109   12766 |  0.97  0.98
--------------------------------------
prec            0.96    0.93    0.98

F1 overall: 0.96
F1 music: 0.96
F1 voice: 0.94
F1 environment: 0.97

####################################
#
# 20180820_orgG527_dropout_450_400_350_300_250_200_150_100_50_21ep4
# mic4
# testData_G428
#
####################################

                   m       v       e |  sens   acc
music          19480    1118      41 |  0.94  0.96
voice            285   13853     298 |  0.96  0.96
environment      671      76   12478 |  0.94  0.98
--------------------------------------
prec            0.95    0.92    0.97

F1 overall: 0.95
F1 music: 0.95
F1 voice: 0.94
F1 environment: 0.96

####################################
#
# 20180820_orgG527_dropout_450_400_350_300_250_200_150_100_50_21ep5
# mic4
# testData_G428


                   m       v       e |  sens   acc
music           8835      80       2 |  0.99  0.98
voice            239    2626       1 |  0.92  0.98
environment        6       2    2483 |  1.00  1.00
--------------------------------------
prec            0.97    0.97    1.00

F1 overall: 0.97
F1 music: 0.98
F1 voice: 0.94
F1 environment: 1.00

####################################
#
# 20180820_orgG527_dropout_450_400_350_300_250_200_150_100_50_21ep1
# mic4
# testData_G527
#
####################################

                   m       v       e |  sens   acc
music           8739     175       3 |  0.98  0.98
voice            121    2738       7 |  0.96  0.98
environment       11       9    2471 |  0.99  1.00
--------------------------------------
prec            0.99    0.94    1.00

F1 overall: 0.97
F1 music: 0.98
F1 voice: 0.95
F1 environment: 0.99

####################################
#
# 20180820_orgG527_dropout_450_400_350_300_250_200_150_100_50_21ep2
# mic4
# testData_G527


                   m       v       e |  sens   acc
music          46433   14268    2894 |  0.73  0.88
voice            189   34935    2780 |  0.92  0.88
environment       12     376   41949 |  0.99  0.96
--------------------------------------
prec            1.00    0.70    0.88

F1 overall: 0.86
F1 music: 0.84
F1 voice: 0.80
F1 environment: 0.93

####################################
#
# 20180820_orgG527_dropout_400_300_200_100_50_20_10ep3
# mic4
# testData_Studio
#
####################################

                   m       v       e |  sens   acc
music          50288   11463    1844 |  0.79  0.90
voice           1260   34567    2077 |  0.91  0.89
environment       19     908   41410 |  0.98  0.97
--------------------------------------
prec            0.98    0.74    0.91

F1 overall: 0.88
F1 music: 0.87
F1 voice: 0.81
F1 environment: 0.94

####################################
#
# 20180820_orgG527_dropout_400_300_200_100_50_20_10ep4
# mic4
# testData_Studio
#
####################

### trainen van model obv: Studio

In [25]:
baseModelFilename = '20180820_orgStudio_dropout'
modelFilePath = storageFolder
logPrefix = 'Orgs Studio met Dropout'
orgWavDirs = orgsStudio
layersList = [[100, 20], [400, 250, 100, 20], [400, 300, 200, 100, 50, 20, 10], 
              [450, 400, 350, 300, 250, 200, 150, 100, 50, 21]]
nrEpochs = 5
nrsEpochs = [1,2,3,4,5]

In [26]:
# Train voor alle lagen
X_train, Y_train, realTrainClasses = getTrainDataFromFolders(orgWavDirs)
for layers in layersList:
    LOG_HEADER(logPrefix + ', lagen: {}'.format(str(layers)), True)
    train_and_evaluate_per_epoch(X_train, Y_train, realTrainClasses, layers, nrEpochs, modelFilePath, baseModelFilename)

####################################
#
# Orgs Studio met Dropout, lagen: [100, 20]
#
####################################


*****************
* Epoch nr 1
*****************

Epoch 1/1
353976/353976 [==============================] - 236s 667us/step - loss: 0.2517 - acc: 0.9036
                   m       v       e |  sens   acc
music         112707    9909     274 |  0.92  0.96
voice           2619  114643    5214 |  0.94  0.94
environment      224    4215  104171 |  0.96  0.97
--------------------------------------
prec            0.98    0.89    0.95

F1 overall: 0.94
F1 music: 0.95
F1 voice: 0.91
F1 environment: 0.95


*****************
* Epoch nr 2
*****************

Epoch 1/1
353976/353976 [==============================] - 217s 613us/step - loss: 0.1758 - acc: 0.9346
                   m       v       e |  sens   acc
music         112294   10475     121 |  0.91  0.97
voice           1195  117622    3659 |  0.96  0.94
environment      214    4419  103977 |  0.96  0.98
-------------

353976/353976 [==============================] - 372s 1ms/step - loss: 0.2501 - acc: 0.9096
                   m       v       e |  sens   acc
music         109260   13454     176 |  0.89  0.96
voice           1008  115531    5937 |  0.94  0.93
environment      346    2936  105328 |  0.97  0.97
--------------------------------------
prec            0.99    0.88    0.95

F1 overall: 0.93
F1 music: 0.94
F1 voice: 0.91
F1 environment: 0.96


*****************
* Epoch nr 2
*****************

Epoch 1/1
353976/353976 [==============================] - 347s 980us/step - loss: 0.1890 - acc: 0.9333
                   m       v       e |  sens   acc
music         114280    8444     166 |  0.93  0.97
voice           1962  118258    2256 |  0.97  0.94
environment      223    7063  101324 |  0.93  0.97
--------------------------------------
prec            0.98    0.88    0.98

F1 overall: 0.94
F1 music: 0.95
F1 voice: 0.92
F1 environment: 0.95


*****************
* Epoch nr 3
*****************

Ep

In [27]:
# Test alle modellen

# model parameters
modelFilePath = storageFolder
layerss = layersList

# test data
micNr = 4
testFileNames = ['testData_G428', 'testData_G527', 'testData_Studio']
keyname = 'mic{}'.format(micNr)

for testFileName in testFileNames:
    for layers in layerss:
        for nrEpochs in nrsEpochs:
            testSpecs, testClasses = retrieveTestData(testFileName, keyname)

            LOG_HEADER(getModelName(baseModelFilename, layers, nrEpochs) 
                       + '\n# ' + keyname
                       + '\n# ' + testFileName,
                       True)
            matrix = evaluate_model(testSpecs, testClasses, layers, nrEpochs, modelFilePath, baseModelFilename)


####################################
#
# 20180820_orgStudio_dropout_100_20ep1
# mic4
# testData_G428
#
####################################

                   m       v       e |  sens   acc
music          20479     154       6 |  0.99  0.86
voice           4564    9645     227 |  0.67  0.90
environment     2029      13   11183 |  0.85  0.95
--------------------------------------
prec            0.76    0.98    0.98

F1 overall: 0.85
F1 music: 0.86
F1 voice: 0.80
F1 environment: 0.91

####################################
#
# 20180820_orgStudio_dropout_100_20ep2
# mic4
# testData_G428
#
####################################

                   m       v       e |  sens   acc
music          20413     225       1 |  0.99  0.84
voice           3929   10361     146 |  0.72  0.91
environment     3787      11    9427 |  0.71  0.92
--------------------------------------
prec            0.73    0.98    0.98

F1 overall: 0.83
F1 music: 0.84
F1 voice: 0.83
F1 environment: 0.83

##################

                   m       v       e |  sens   acc
music          20411     216      12 |  0.99  0.88
voice           3679   10549     208 |  0.73  0.91
environment     1983      11   11231 |  0.85  0.95
--------------------------------------
prec            0.78    0.98    0.98

F1 overall: 0.87
F1 music: 0.87
F1 voice: 0.84
F1 environment: 0.91

####################################
#
# 20180820_orgStudio_dropout_450_400_350_300_250_200_150_100_50_21ep4
# mic4
# testData_G428
#
####################################

                   m       v       e |  sens   acc
music          20360     277       2 |  0.99  0.85
voice           3697   10581     158 |  0.73  0.91
environment     3479      16    9730 |  0.74  0.92
--------------------------------------
prec            0.74    0.97    0.98

F1 overall: 0.84
F1 music: 0.85
F1 voice: 0.84
F1 environment: 0.84

####################################
#
# 20180820_orgStudio_dropout_450_400_350_300_250_200_150_100_50_21ep5
# mic4
# testData_G

                   m       v       e |  sens   acc
music           8912       4       1 |  1.00  0.90
voice            653    2210       3 |  0.77  0.95
environment      720       0    1771 |  0.71  0.95
--------------------------------------
prec            0.87    1.00    1.00

F1 overall: 0.88
F1 music: 0.93
F1 voice: 0.87
F1 environment: 0.83

####################################
#
# 20180820_orgStudio_dropout_450_400_350_300_250_200_150_100_50_21ep1
# mic4
# testData_G527
#
####################################

                   m       v       e |  sens   acc
music           8867      49       1 |  0.99  0.91
voice            627    2239       0 |  0.78  0.95
environment      594       0    1897 |  0.76  0.96
--------------------------------------
prec            0.88    0.98    1.00

F1 overall: 0.89
F1 music: 0.93
F1 voice: 0.87
F1 environment: 0.86

####################################
#
# 20180820_orgStudio_dropout_450_400_350_300_250_200_150_100_50_21ep2
# mic4
# testData_G

####################################
#
# 20180820_orgStudio_dropout_400_300_200_100_50_20_10ep2
# mic4
# testData_Studio
#
####################################

                   m       v       e |  sens   acc
music          59284    4226      85 |  0.93  0.96
voice            647   36665     592 |  0.97  0.96
environment      188     397   41752 |  0.99  0.99
--------------------------------------
prec            0.99    0.89    0.98

F1 overall: 0.96
F1 music: 0.96
F1 voice: 0.93
F1 environment: 0.99

####################################
#
# 20180820_orgStudio_dropout_400_300_200_100_50_20_10ep3
# mic4
# testData_Studio
#
####################################

                   m       v       e |  sens   acc
music          58161    5309     125 |  0.91  0.96
voice            282   37169     453 |  0.98  0.96
environment       83     383   41871 |  0.99  0.99
--------------------------------------
prec            0.99    0.87    0.99

F1 overall: 0.95
F1 music: 0.95
F1 voice: 0.92
